In [1]:
import os
import cv2
from sklearn.model_selection import train_test_split

In [2]:

def read_videos_from_directory(directory):
    video_files = [f for f in os.listdir(directory) if f.endswith('.mp4') or f.endswith('.avi')]
    videos = []
    for video_file in video_files:
        video_path = os.path.join(directory, video_file)
        videos.append(video_path)
    return videos


In [3]:
def extract_frames(video_path, output_directory):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Read and save each frame
    frame_count = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Save the frame as an image file
        frame_filename = os.path.join(output_directory, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # Release the video capture object
    cap.release()

In [4]:
def split_and_extract(video_folder, output_folder, test_size=0.2):
    # Get all video files in the specified folder
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4') or f.endswith('.avi')]

    # Split videos into train and test sets
    train_videos, test_videos = train_test_split(video_files, test_size=test_size, random_state=42)

    # Process train videos
    for video_file in train_videos:
        video_path = os.path.join(video_folder, video_file)
        output_directory = os.path.join(output_folder, 'train', os.path.splitext(video_file)[0])
        extract_frames(video_path, output_directory)
        print(f"Frames extracted for {video_file} to {output_directory}")

    # Process test videos
    for video_file in test_videos:
        video_path = os.path.join(video_folder, video_file)
        output_directory = os.path.join(output_folder, 'test', os.path.splitext(video_file)[0])
        extract_frames(video_path, output_directory)
        print(f"Frames extracted for {video_file} to {output_directory}")

if __name__ == "__main__":
    # Replace 'input_videos_folder' and 'output_frames_folder' with your folder paths
    input_videos_folder = 'Output/'
    output_frames_folder = 'OutputFrames'

    # Create 'train' and 'test' subdirectories
    train_output_folder = os.path.join(output_frames_folder, 'train')
    test_output_folder = os.path.join(output_frames_folder, 'test')

    if not os.path.exists(train_output_folder):
        os.makedirs(train_output_folder)

    if not os.path.exists(test_output_folder):
        os.makedirs(test_output_folder)

    split_and_extract(input_videos_folder, output_frames_folder)

Frames extracted for walking1.mp4 to OutputFrames/train/walking1
Frames extracted for walking.mp4 to OutputFrames/train/walking
Frames extracted for Harassing1.mp4 to OutputFrames/train/Harassing1
Frames extracted for Fighting.mp4 to OutputFrames/train/Fighting
Frames extracted for walking2.mp4 to OutputFrames/train/walking2
Frames extracted for Helping.mp4 to OutputFrames/train/Helping
Frames extracted for Fighting1.mp4 to OutputFrames/train/Fighting1
Frames extracted for Harass3.mp4 to OutputFrames/train/Harass3
Frames extracted for Sitting.mp4 to OutputFrames/train/Sitting
Frames extracted for Sitting1.mp4 to OutputFrames/train/Sitting1
Frames extracted for Harassing.mp4 to OutputFrames/test/Harassing
Frames extracted for Harass2.mp4 to OutputFrames/test/Harass2
Frames extracted for Running.mp4 to OutputFrames/test/Running


In [5]:
!pip install keras

In [6]:
!pip install Keras-Preprocessing

In [7]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [8]:
TRAIN_DIR = 'OutputFrames/train'
TEST_DIR = 'OutputFrames/test'

In [9]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [10]:
# Function to extract features from a directory of images
def extract_features_from_directory(directory, model):
    features_list = []
    labels = []

    for class_label in os.listdir(directory):
        class_path = os.path.join(directory, class_label)

        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            features = extract_features(img_path, model)
            features_list.append(features)
            labels.append(class_label)

    return features_list, labels

In [11]:
# Function to extract features from an image
def extract_features(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))  # Resize image to fit model input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess input according to ResNet50 requirements
    features = model.predict(img_array)
    return features.flatten()  # Flatten the features to create a 1D array

In [12]:
def load_images(file_paths):
    return [cv2.imread(file_path) for file_path in file_paths]

In [13]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [14]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

Harassing1 completed
Helping completed
walking2 completed
Harass3 completed
walking completed
Fighting completed
walking1 completed
Sitting completed
Fighting1 completed
Sitting1 completed


In [15]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

Running completed
Harass2 completed
Harassing completed


In [16]:
from tqdm.notebook import tqdm

In [17]:
from keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import numpy as np

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = img_to_array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [18]:
train_features = extract_features(train['image'])

100%|████████████████████████████████████| 34342/34342 [01:57<00:00, 292.53it/s]


In [19]:
test_features = extract_features(test['image'])

100%|██████████████████████████████████████| 8453/8453 [00:29<00:00, 287.12it/s]


In [20]:
x_train = train_features/255.0
x_test = test_features/255.0

In [21]:
from sklearn.preprocessing import LabelEncoder

In [25]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [26]:
y_train = le.transform(train['label'])
#y_test = le.transform(test['label'])

In [27]:
# Split the data into training and testing sets
X = x_train
y = y_train
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes to verify
print("X_train Shape:", x_train.shape)
print("X_test Shape:", x_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:", y_test.shape)

# Use LabelEncoder for encoding the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


X_train Shape: (27473, 48, 48, 1)
X_test Shape: (6869, 48, 48, 1)
y_train Shape: (27473,)
y_test Shape: (6869,)


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define a simple neural network model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification, adjust for your task

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy}')

Epoch 1/30
687/687 [==============================] - 4s 6ms/step - loss: -6687537.0000 - accuracy: 0.1137 - val_loss: -27022494.0000 - val_accuracy: 0.1172
Epoch 2/30
687/687 [==============================] - 4s 6ms/step - loss: -95919248.0000 - accuracy: 0.1137 - val_loss: -197376848.0000 - val_accuracy: 0.1172
Epoch 3/30
687/687 [==============================] - 4s 6ms/step - loss: -372557792.0000 - accuracy: 0.1137 - val_loss: -590070464.0000 - val_accuracy: 0.1172
Epoch 4/30
687/687 [==============================] - 4s 6ms/step - loss: -894390016.0000 - accuracy: 0.1137 - val_loss: -1246860160.0000 - val_accuracy: 0.1172
Epoch 5/30
687/687 [==============================] - 4s 6ms/step - loss: -1694927616.0000 - accuracy: 0.1137 - val_loss: -2194767360.0000 - val_accuracy: 0.1172
Epoch 6/30
687/687 [==============================] - 5s 7ms/step - loss: -2797867264.0000 - accuracy: 0.1137 - val_loss: -3457641984.0000 - val_accuracy: 0.1172
Epoch 7/30
687/687 [===================

In [33]:

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 16928)             0         
                                                                 
 dense_8 (Dense)             (None, 64)                1083456   
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1083841 (4.13 MB)
Trainable params: 1083841 (4.13 MB)
Non-trainable params: 0 (0.00 Byte)
________________